 ##  Segmenting and Clustering Neighborhoods in Toronto

This is for assignment Segmenting and Clustering Neighborhoods in Toronto

**Created by: Charles Fung**  11/2018

### Part 1 DataFrame of Neighborhools in Toronto by web scraping from Wikipedia

#### Requirements from lab assignment web page in Coursera:

1. Scrape Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
1. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
1. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

##### Assumptions:
1. When two neighborhoods are combined, they are separated with a comma **ONLY**. The assignment web page in Coursera shows a space following the comma. This is not significant as following code does not use the neighborhood names individually.
1. When two neighborhoods are combined, their names are not ordered. The assignment web page in Coursera does not show them in any specific order. The script below preserves the same order as they appear in the Wikipedia page.
1. Assumed that the names in the Wikipedia page table are consistent in spelling. For example, 'East Toronto' and 'east Toronto' will be treated as different boroughs.
1. Some of the Neighborhools in the result DataFrame actually do not belong but we will leave them in, e.g.
    - M5W	Downtown Toronto	"Stn A PO Boxes 25 The Esplanade" (specific Postal Boxes for mail processing?)
    - M7Y	East Toronto	"Business reply mail Processing Centre969 Eastern"  (specific facility?)
    - M7R	Mississauga	"Canada Post Gateway Processing Centre" (Mississauga is officially not part of Toronto)

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

# uncomment the line to install beautifulsoup4
#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # library to parse html

print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser') # create the soup, using basic html parser
table = soup.find('table', class_='wikitable') # access the table by its class_

In [3]:
# iterate the rows and cells in the table
rows = []
prev_row = [] # previous row
for row in table.find_all('tr'):
    cells = []
    for cell in row.find_all('td'):
        cells.append(cell.text.strip('\n')) # capture cell text and remove trailing \n
    if len(cells) and (cells[1]!="Not assigned"):
        # if Neighborhood is Not Assigned, use the same from Borough
        if cells[2] == "Not assigned":
            cells[2] = cells[1]
        # if Postal Code and Borough in this row and the previous row match, concatenate the Neighborhood
        # and put into the previous row
        if len(prev_row) and (cells[0:2] == prev_row[0:2]):
            prev_row[2] = ','.join([prev_row[2], cells[2]])
        else:
            prev_row = cells
            rows.append(cells)

rows[0:5] #sanity check            

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront,Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights,Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"]]

In [4]:
df = pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head() #sanity check

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
df.Borough.value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [6]:
print("There are {} rows in the DataFrame".format(df.shape[0]))

There are 103 rows in the DataFrame


### Part 2 Get latitude and longitude coordinates of each neighborhood

#### We could have used geocoder to, but the service was not responsive , so I opted to use the downloaded coordinates file.

In [7]:
# download Geospatial_Coordinates.csv if needded
#!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

In [8]:
# read coordinates file
df_geospatial = pd.read_csv("Geospatial_Coordinates.csv")

In [9]:
df_geospatial.head()  #sanity check

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# merge with df and drop the extra 'Postal Code' column
df2 = df.merge(df_geospatial, how='inner', left_on="PostalCode", right_on="Postal Code").drop("Postal Code", axis = 1)

In [11]:
df2.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [12]:
df2.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


### Part 3 Exploring Toronto neighborhoods

In [13]:
import numpy as np # library to handle data in a vectorized manner
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# uncomment this line if you need to install
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't installed folium
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


**Let's select Boroughs that contain 'Toronto' in their names, and the five boroughs prior to the 1988 amalgamation into Toronto: East York, North York, Scarborough, Etobicoke, York. This removes, for example, Mississauga which is outside Toronto.**

In [14]:
df3a = df2[df2.Borough.str.contains("Toronto")].reset_index(drop=True)
df3b = df2[df2.Borough.isin(["East York", "North York", "Scarborough", "Etobicoke", "York"])].reset_index(drop=True)
df3 = pd.concat([df3a, df3b]).reset_index(drop=True)

In [15]:
df3.shape

(101, 5)

In [16]:
df3 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df3['Borough'].unique()),
        df3.shape[0]
    )
)

The dataframe has 9 boroughs and 101 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.

In [18]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="my-coursera-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Some observations
##### About 10 neighborhoods are packed at Downtown Toronto. Remember that the neighborhoods are based on mailing postal codes, which are tended to be proportional to population density. An alterative option is to use the 25 city council wards as neighborhoods.
##### Another observation is that the neighborhood coordinates are randomly picked. Some of them fall into industrial area (e.g. Agincourt, Scarborough), park (e.g. Scarborough Village, Scarborough), racing course (Northwest, Etobicoke). It is highly possible there are not many venues within 500m of these coordinates.
##### A possible alternative is to use Tim Horton's coffee shop or MacDonald's outlets in each neighborhoods as coordinates for venue search.

#### Define Foursquare Credentials and Version

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'hidden' # your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100

#### A function to retrieve venues near a collection of coordinates

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    """
    Retrieve venues near to coordinates using foursquare API
    
    Arguments:
    names - names of neighborhood, list of string
    latitudes - latitudes of neighborhood, list of float
    longitudes - longitudes of neighborhood, list of float
    radius - search limit in m, int
    
    Return:
    pandas.DataFrame containing neighborhood and venue information: latitudes, longitudes, catefories
    """
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

In [24]:
# get venues for Toronto neighborhoods
toronto_venues = getNearbyVenues(df3.Neighborhood, df3.Latitude, df3.Longitude)
print("Call completed.")

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [25]:
print(toronto_venues.shape)
toronto_venues.head()  #sanity check

(2199, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Let's check how many venues were returned for each neighborhood

In [26]:
toronto_venues_grouped = toronto_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
toronto_venues_grouped.index.names = ['Neighborhood']
toronto_venues_grouped.reset_index('Neighborhood', inplace=True)
toronto_venues_grouped

,Neighborhood,Venue
0,"Adelaide,King,Richmond",100
1,Agincourt,4
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",2
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",9
4,"Alderwood,Long Branch",11
5,"Bathurst Manor,Downsview North,Wilson Heights",17
6,Bayview Village,4
7,"Bedford Park,Lawrence Manor East",24
8,Berczy Park,55
9,"Birch Cliff,Cliffside West",4


In [27]:
toronto_venues_grouped.sort_values('Venue', ascending=False, inplace=True)
toronto_venues_grouped.reset_index(inplace=True, drop=True)
toronto_venues_grouped.head(20)

,Neighborhood,Venue
0,"Adelaide,King,Richmond",100
1,St. James Town,100
2,"Ryerson,Garden District",100
3,"Harbourfront East,Toronto Islands,Union Station",100
4,"First Canadian Place,Underground city",100
5,"Design Exchange,Toronto Dominion Centre",100
6,"Commerce Court,Victoria Hotel",100
7,"Chinatown,Grange Park,Kensington Market",100
8,Stn A PO Boxes 25 The Esplanade,93
9,Church and Wellesley,88


In [28]:
toronto_venues_grouped.shape

(99, 2)

In [29]:
toronto_venues.shape

(2199, 7)

### Observations
##### There are 2 neighborhoods with 0 venues found, and they are left out from toronto_venues_grouped.
##### 15 neighborhoods have over 50 venues found. The other 84 neighborhoods have less than 50 venues found.

#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


### Analyze Each Neighborhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
# the new column does not always append to the end, so figure out where it is and concatenate accordingly
idx = toronto_onehot.columns.tolist().index('Neighborhood')
fixed_columns = ([toronto_onehot.columns[idx]] + list(toronto_onehot.columns[0:idx]) + list(toronto_onehot.columns[idx+1:]))
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Wo

In [32]:
# examine the shape
toronto_onehot.shape

(2199, 280)

### Group rows by neighborhood and take the mean of the frequency of occurrence of each category.

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Wo

In [34]:
# examine the shape
toronto_grouped.shape

(99, 280)

### Print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4                  Bar  0.03


----Agincourt----
            venue  freq
0  Clothing Store  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25
3    Skating Rink  0.25
4     Music Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                        venue  freq
0                  Playground   0.5
1                        Park   0.5
2  Modern European Restaurant   0.0
3                 Music Store   0.0
4                      Museum   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1           Pizza Place  0.11
2            Beer Store  0.11
3           Coffee Shop  0.11
4  Fast Food Restaurant  0.11


----Alderwood,Long Branch----
          venue  freq
0   

                             venue  freq
0                Convenience Store  0.33
1                      Coffee Shop  0.33
2                             Park  0.33
3                Accessories Store  0.00
4  Molecular Gastronomy Restaurant  0.00


----Emery,Humberlea----
                           venue  freq
0  Paper / Office Supplies Store   0.5
1                 Baseball Field   0.5
2        New American Restaurant   0.0
3                    Music Store   0.0
4                         Museum   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.15
1  Fast Food Restaurant  0.07
2           Coffee Shop  0.06
3         Metro Station  0.03
4      Asian Restaurant  0.03


----First Canadian Place,Underground city----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.08
2        Hotel  0.06
3   Restaurant  0.05
4   Steakhouse  0.04


----Flemingdon Park,Don Mills South----
                 venue  freq
0          Coffee Shop  0.10
1  

                venue  freq
0                Café  0.08
1         Pizza Place  0.08
2         Coffee Shop  0.08
3            Tea Room  0.05
4  Italian Restaurant  0.05


----Ryerson,Garden District----
                       venue  freq
0                Coffee Shop  0.07
1             Clothing Store  0.07
2                       Café  0.04
3         Italian Restaurant  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                        venue  freq
0                  Playground  0.33
1                         Spa  0.33
2  Construction & Landscaping  0.33
3                    Hospital  0.00
4                   Nightclub  0.00


----Silver Hills,York Mills----
                 venue  freq
0    Martial Arts Dojo   0.5
1            Cafeteria   0.5
2    Accessories Store   0.0
3  Monument / Landmark   0.0
4          Music Venue   0.0


----St. James Town----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2   Restaurant  0.05
3        Hotel  0.04
4   

### Observations
##### If a neighborhood has only a few venues found (e.g. Newtonbrook,Willowdale has only one venue, so Gym = 1.0), its frequency distribution is greatly skewed.

###### We could see that 'Coffee Shop'  and 'Cafe' are very popular in the categories. It means that they are present in most neighborhoods and do not help to distinguish them in clustering. We may remove 'Coffee Shop' and 'Cafe' from the venues.

In [36]:
#toronto_onehot_revised = toronto_onehot.drop(['Coffee Shop', 'Café'], axis= 1)
#toronto_onehot_revised.head()

In [37]:
#toronto_grouped_revised = toronto_onehot_revised.groupby('Neighborhood').mean().reset_index()
#toronto_grouped_revised.head(10)

In [38]:
#toronto_grouped = toronto_grouped_revised

### Put the above into a pandas dataframe

#### Write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    """
    Get the most common venues from a pandas DataFrame
    
    Arguments:
    row - data with venues frequency in columns 1 to end
    num_top_venues - number of top venues, int
    
    Return:
    DataFrame of shape(1, num_top_venues). Each column value is a venue(string). Column 1 is most common, and so on
    
    """
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues for each neighborhood.

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to ordinal number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Breakfast Spot,Hotel,Thai Restaurant,Restaurant,Bar,Gym
1,Agincourt,Lounge,Clothing Store,Breakfast Spot,Skating Rink,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Coffee Shop,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,Event Space,Ethiopian Restaurant
4,"Alderwood,Long Branch",Pizza Place,Bank,Coffee Shop,Pool,Pub,Dance Studio,Sandwich Place,Skating Rink,Pharmacy,Gym
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pizza Place,Chinese Restaurant,Bridal Shop,Shopping Mall,Sandwich Place,Bank,Diner,Sushi Restaurant,Fried Chicken Joint
6,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Pharmacy,Ice Cream Shop,Pizza Place,Pub,Restaurant,Café,Butcher
8,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Farmers Market,Seafood Restaurant,Café,Pub,Steakhouse
9,"Birch Cliff,Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0])

In [42]:
len(kmeans.labels_)

99

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
toronto_merged = pd.DataFrame(toronto_grouped['Neighborhood']).set_index('Neighborhood')

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_data = df3[["Borough", "Neighborhood", "Latitude", "Longitude"]].set_index('Neighborhood')
toronto_merged = toronto_data.join(toronto_merged, how="inner", on='Neighborhood')

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.reset_index(inplace=True)
# move Borough column to the front
columns = [toronto_merged.columns[1]] + [toronto_merged.columns[0]] + list(toronto_merged.columns[2:5]) + list(toronto_merged.columns[5:])
toronto_merged = toronto_merged[columns]
toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Electronics Store,Brewery
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Theater,Burger Joint,Bubble Tea Shop
2,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Gastropub,Clothing Store,Cocktail Bar,Bakery,Cosmetics Shop,Park
3,East Toronto,The Beaches,43.676357,-79.293031,0,Boutique,Coffee Shop,Pub,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Farmers Market,Seafood Restaurant,Café,Pub,Steakhouse


In [44]:
toronto_merged['Cluster Labels'].value_counts()

0    88
2     6
3     3
4     1
1     1
Name: Cluster Labels, dtype: int64

#### Visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Mexican Restaurant,Electronics Store,Brewery
1,"Ryerson,Garden District",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Theater,Burger Joint,Bubble Tea Shop
2,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Gastropub,Clothing Store,Cocktail Bar,Bakery,Cosmetics Shop,Park
3,The Beaches,Boutique,Coffee Shop,Pub,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Farmers Market,Seafood Restaurant,Café,Pub,Steakhouse
5,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Japanese Restaurant,Chinese Restaurant,Burger Joint,Bubble Tea Shop,Sandwich Place,Middle Eastern Restaurant
6,Christie,Café,Grocery Store,Park,Italian Restaurant,Diner,Athletics & Sports,Restaurant,Nightclub,Baby Store,Coffee Shop
7,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Breakfast Spot,Hotel,Thai Restaurant,Restaurant,Bar,Gym
8,"Dovercourt Village,Dufferin",Pharmacy,Supermarket,Bakery,Park,Music Venue,Brewery,Liquor Store,Middle Eastern Restaurant,Bar,Bank
9,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Sports Bar,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"Rouge,Malvern",Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Rosedale,Park,Trail,Playground,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
72,"Newtonbrook,Willowdale",Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
81,Weston,Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
83,York Mills West,Park,Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
90,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
96,"The Kingsway,Montgomery Road,Old Mill North",River,Park,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,Downsview Central,Baseball Field,Food Truck,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio
76,"Emery,Humberlea",Baseball Field,Paper / Office Supplies Store,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio
97,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Baseball Field,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio,Discount Store


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,"Highland Creek,Rouge Hill,Port Union",Bar,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Discount Store


### Observations
##### Cluster 0 has most of the neighborhoods. In addition to Coffee Shop and Cafe, they show a good mix of restaurants and facilities. Their distribution on map just shows that Toronto is very well-mixed and homogeneous
#### An issue with foursquare API is that users tend to record a lot of restaurants as venues but there is not a way to categorize the cuisines systematically. This makes data understanding a lot more difficult.

##### Clusters 1 and 2: Scarborough Village and Newtonbrook, Willowdale are both having only one venue available. K-means clustering has put them into their own cluster, but it does not really make sense.
##### A possible enhancement is to remove the neighborhoods with less than say, 20, venues from foursquare API.
##### Cluster 3 include Dog Run, Donut Shop and Drugstore. These neighborhoods seem to be suitable for aged residents.
##### Cluster 4 include Park, Playground, Yoga Studio etc. These neighborhoods seem to be the healthy ones!